In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,concatenate,AveragePooling2D,Dropout,Dense,Softmax
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow_datasets as tfds
import cv2
import numpy as np

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'data/train',
    seed=123,
    batch_size=32,
    image_size=(224,224))


test_ds = tf.keras.utils.image_dataset_from_directory(
    'data/test/',
    seed=123,
    image_size=(224,224))

val_ds = tf.keras.utils.image_dataset_from_directory(
    'data/valid/',
    seed=123,
    image_size=(224,224))


Found 2339 files belonging to 10 classes.
Found 50 files belonging to 10 classes.
Found 50 files belonging to 10 classes.


In [11]:
train_ds.class_names

['AFRICAN LEOPARD',
 'CARACAL',
 'CHEETAH',
 'CLOUDED LEOPARD',
 'JAGUAR',
 'LIONS',
 'OCELOT',
 'PUMA',
 'SNOW LEOPARD',
 'TIGER']

In [3]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
        super(InceptionModule, self).__init__()
        
        self.conv1 = Conv2D(filters_1x1, kernel_size=(1, 1), padding='same', activation='relu')
        
        self.conv2_reduce = Conv2D(filters_3x3_reduce, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2 = Conv2D(filters_3x3, kernel_size=(3, 3), padding='same', activation='relu')
        
        self.conv3_reduce = Conv2D(filters_5x5_reduce, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv3 = Conv2D(filters_5x5, kernel_size=(5, 5), padding='same', activation='relu')
        
        self.pool_proj = Conv2D(filters_pool_proj, kernel_size=(1, 1), padding='same', activation='relu')
        
    def call(self, inputs):
        conv1_out = self.conv1(inputs)
        
        conv2_reduce_out = self.conv2_reduce(inputs)
        conv2_out = self.conv2(conv2_reduce_out)
        
        conv3_reduce_out = self.conv3_reduce(inputs)
        conv3_out = self.conv3(conv3_reduce_out)
        
        pool_out = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)
        pool_proj_out = self.pool_proj(pool_out)
        
        inception_out = concatenate([conv1_out, conv2_out, conv3_out, pool_proj_out], axis=-1)
        
        return inception_out
    
class GoogleNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(GoogleNet, self).__init__()
        
        self.conv1 = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')
        self.max_pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.conv2_reduce = Conv2D(64, kernel_size=(1, 1), padding='same', activation='relu')
        self.conv2 = Conv2D(192, kernel_size=(3, 3), padding='same', activation='relu')
        self.max_pool2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.inception3a = InceptionModule(64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionModule(128, 128, 192, 32, 96, 64)
        self.max_pool3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.inception4a = InceptionModule(192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionModule(160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionModule(128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionModule(112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionModule(256, 160, 320, 32, 128, 128)
        self.max_pool4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        
        self.inception5a = InceptionModule(256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionModule(384, 192, 384, 48, 128, 128)
        
        self.avg_pool = AveragePooling2D(pool_size=(7, 7), strides=(1, 1), padding='valid')
        self.dropout = Dropout(0.4)
        self.fc1 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.max_pool1(x)
        
        x = self.conv2_reduce(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.max_pool3(x)
        
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.max_pool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avg_pool(x)
        x = self.dropout(x)
        
        return self.fc1(x)

In [4]:
model = GoogleNet(num_classes=10)

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [10]:
EPOCHS = 5
with tf.device('/device:GPU:0'):
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

c:\Users\justf\miniconda3\envs\gpu\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Loss: 2.4024038314819336, Accuracy: 9.821966171264648, Test Loss: 2.300820827484131, Test Accuracy: 10.830860137939453
Epoch 2, Loss: 2.315138101577759, Accuracy: 9.989165306091309, Test Loss: 2.3193717002868652, Test Accuracy: 10.830860137939453
Epoch 3, Loss: 2.3072822093963623, Accuracy: 9.603938102722168, Test Loss: 2.3045852184295654, Test Accuracy: 11.869436264038086
Epoch 4, Loss: 2.305021047592163, Accuracy: 9.090301513671875, Test Loss: 2.301018714904785, Test Accuracy: 7.7151336669921875
Epoch 5, Loss: 2.3039932250976562, Accuracy: 9.304316520690918, Test Loss: 2.3027234077453613, Test Accuracy: 9.792284965515137
